<a href="https://colab.research.google.com/github/raketic-ognjen/Market-Watch-Tech-Analysis/blob/main/TechAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = "DeepSkyBlue"><center>**Market Watch - Technical Analysis**</center></font>

### <center>**Core Idea Explanation!**</center>

### Adding Packages - Libraries:
1. Pandas(pandas data reader)
2. yfinance
3. NumPy(linAlg)
4. SciPy(optimize and stats)
5. Plotly(graph objects and subplots)
6. Statsmodels.api
7. DateTime

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import numpy.linalg as LA
import scipy as sp
import scipy.optimize as sco
import scipy.stats as st
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
import pandas_datareader as pdr
import datetime as dt
import MCF_INV as inv

> First of all, I want to appologise to my team and to You, professor, because I did not show up on the Market Watch Game due to my **TCL knee injury.** Because of that I will do next things:
1.   **Portfolio Optimization**
2.   **Efficient Frontiers**
3.   **CAPM Model**
4.   **Sharpe's Ratio**
5.   **Testing Portfolio Performances**
6.   **Robustness portfolio**
7.   **Black-Litterman model**
8.   **Comparing with Non-Technical Result of my group**



### <center>*Work*</center>

According to my group starting portfolio, we use 6 companies:
1. Starbucks - SBUX - 9 stocks 0.13 weight
2. Coca Cola - KO - 15  stocks 0.22 weight
3. Microsoft - MSFT 3 stocks 0.04 weight
4. Pfizer - PFE 25 stocks 0.37 weight
5. Nike - NKE 8 stocks 0.12 weight
6. Exxon Mobil - XOM 8 stocks 0.12 weight

Later Non Technical portfolio  had 83 trades and a lot buying-selling companies, so for this analysis I will use only this 6 stocks

According to TA starting portfolio, we use 6 companies:
1. Astra Zeneca - AZN -  
2. Nvidia - NVD -
3. Novartis - NVS
4. Rivian - RIVN
5. Carnival Corporation - CCL
6. Norwegian Cruise Company -NCLH

## NTA Portfolio

In [ ]:
#Non Technical Analysis Portfolio start
ntaportf= ['SBUX', 'KO', 'MSFT','PFE', 'NKE', 'XOM']

### Preparation data for optimization of NTA portfolio

In [ ]:
ntaprices = yf.download(ntaportf,start='2022-05-18',end='2023-05-18')['Adj Close']
#ntaprices.to_csv('ntaprices.csv',encoding='utf-8-sig')
#files.download('ntaprices.csv')

[*********************100%%**********************]  6 of 6 completed


In [ ]:
rets1=pd.read_csv('ntaprices.csv',index_col=0)

In [ ]:
#returns on NTA portfolio
rets1=ntaprices.pct_change().dropna()

#Weights for NTA
w1 = [0.13,0.22,0.04,0.37,0.12,0.12]

#Portfolio returns
rp1=rets1@w1
rp1

Date
2022-05-19   -0.000856
2022-05-20    0.018884
2022-05-23    0.016880
2022-05-24    0.004252
2022-05-25    0.006996
                ...   
2023-05-11   -0.007343
2023-05-12   -0.002677
2023-05-15   -0.001188
2023-05-16   -0.006898
2023-05-17    0.002667
Length: 250, dtype: float64

### From portfolio returns (Portfolio as an asset)

In [ ]:
inv.expRet(rets1,True),inv.annualize_vol(rets1,True)

(KO      0.000296
 MSFT    0.001101
 NKE     0.000597
 PFE    -0.001012
 SBUX    0.001834
 XOM     0.000927
 dtype: float64,
 KO      0.010386
 MSFT    0.020836
 NKE     0.023806
 PFE     0.014571
 SBUX    0.018550
 XOM     0.020540
 dtype: float64)

In [ ]:
er1= inv.expRet(rp1)
print(er1)
vol1= inv.annualize_vol(rp1)
print(vol1)

0.06586675191165836
0.19451632312794037


### From stocks' returns (Portfolio as collection of assets)

In [ ]:
per1 = inv.per(W=w1,ret=rets1, n=250)
per1

0.06534399991235948

Computation risk of Portfolio

In [ ]:
pV1=inv.pV(w1,rets1, 250, True)
pV1

0.19374289532087313

### MVP

In [ ]:
mvp1= inv.mvp(ret=rets1,f=250,com=True)
mvp1

{'w': array([ 0.74264428, -0.01524857, -0.08163824,  0.15621429,  0.07963346,
         0.11839479]),
 'er': 0.06202083161480626,
 'vol': 0.15366164058585924}

In [ ]:
mvp1['w']

array([ 0.74264428, -0.01524857, -0.08163824,  0.15621429,  0.07963346,
        0.11839479])

### Manual optimization

In [ ]:
res1=sco.minimize(lambda w1: inv.pV(w1,rets1),
                  [0.13,0.22,0.04,0.37,0.12,0.12],
                  constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1)])
res1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.023800699100657265
       x: [ 7.433e-01 -1.615e-02 -8.157e-02  1.548e-01  8.104e-02
            1.185e-01]
     nit: 15
     jac: [ 4.757e-02  4.712e-02  4.748e-02  4.758e-02  4.759e-02
            4.769e-02]
    nfev: 106
    njev: 15

In [ ]:
target1= inv.targetP(rets1,0.1)
target1

{'w': array([ 0.75048388, -0.00959417, -0.09213333,  0.10127596,  0.12249186,
         0.12747581]),
 'er': 0.1,
 'vol': 0.1548729102313266}

In [ ]:
result1 = sco.minimize(lambda w1: inv.pV(w1,rets1),
                  [0.13,0.22,0.04,0.37,0.12,0.12],
                  constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1),
                               dict(type='eq',fun=lambda w1: inv.per(w1,rets1)-0.08)])
result1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.023837371332415334
       x: [ 7.449e-01 -1.489e-02 -8.362e-02  1.311e-01  9.893e-02
            1.236e-01]
     nit: 11
     jac: [ 4.764e-02  4.821e-02  4.875e-02  4.618e-02  4.970e-02
            4.866e-02]
    nfev: 78
    njev: 11

In [ ]:
resnta = sco.minimize(lambda w1: inv.pV(w1,rets1),
                  [1/6]*6,
                      bounds=[[0,0.04]]*6,
                  constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1),
                               dict(type='eq',fun=lambda w1: inv.per(w1,rets1)-0.1)])
resnta

 message: Iteration limit reached
 success: False
  status: 9
     fun: 0.002452642462460355
       x: [ 4.000e-02  4.000e-02  4.000e-02  4.000e-02  4.000e-02
            4.000e-02]
     nit: 100
     jac: [ 1.083e-02  2.525e-02  3.086e-02  1.440e-02  2.246e-02
            1.884e-02]
    nfev: 1470
    njev: 100

### Efficient Frontier for NTA portfolio

In [ ]:
ntaEF = inv.EF(rets1,plot='full')
ntaEF

In [ ]:
fignta=inv.EF(rets1,Range=[0.01,1.5],plot='full')
fignta

In [ ]:
rfa1 = pdr.get_data_fred('DTB3',start='2022-05-18',end='2023-05-18')/100
rfa1.head()
rfa1.dropna()

,DTB3
DATE,
2022-05-18,0.0102
2022-05-19,0.0104
2022-05-20,0.0101
2022-05-23,0.0107
2022-05-24,0.0105
...,...
2023-05-12,0.0508
2023-05-15,0.0506
2023-05-16,0.0507


### Deannualization(EAR)

In [ ]:
rf1 = (1+rfa1)**(1/251)-1
rf1.head()
rf1.dropna()

,DTB3
DATE,
2022-05-18,0.000040
2022-05-19,0.000041
2022-05-20,0.000040
2022-05-23,0.000042
2022-05-24,0.000042
...,...
2023-05-12,0.000197
2023-05-15,0.000197
2023-05-16,0.000197


In [ ]:
opt1 = inv.maxSharpe(rets1,rf1,f=251,com=True)
opt1

{'w': array([ 0.88610827,  0.27905304, -0.46499815, -1.        ,  1.        ,
         0.29983685]),
 'er': 0.9080784816438078,
 'vol': 0.34115628277183563,
 'sharpe': 2.556557380493319}

In [ ]:
fignta.add_trace(go.Scatter(x=[inv.pV(opt1['w'],rets1,n=250,vol=True)],
                            y=[inv.per(opt1['w'],rets1)],
                            marker=dict(
                                color='Crimson',
                                size=10,
                                line=dict(
                                    color='Black',
                                    width=2
                                )
                            )
                          )
)

#plot show
fignta.show()

Test for Eff frontier

In [ ]:
#Initial Wealth
Wealth1 = 30_000

#Determined amount invested in each stock
si1=opt1['w']*Wealth1

#Number of stocks
n1=si1/ntaprices.iloc[0]
n1

KO      454.205895
MSFT     33.337566
NKE    -129.850485
PFE    -624.034805
SBUX    432.262392
XOM     103.501562
Name: 2022-05-18 00:00:00, dtype: float64

In [ ]:
#Value of portfolio each point in the time
ST1 = ntaprices@n1
ST1

Date
2022-05-18    30000.000000
2022-05-19    29994.894449
2022-05-20    29805.075715
2022-05-23    30777.898528
2022-05-24    30859.259002
                  ...     
2023-05-11    56080.179119
2023-05-12    56765.040536
2023-05-15    57115.665091
2023-05-16    56547.502847
2023-05-17    57122.375817
Length: 251, dtype: float64

### Static Trading Strategy for NTA portfolio

In [ ]:
#preparing data
return1 = ntaprices.iloc[:,:6].pct_change()
return1.dropna(inplace=True)

#initial wealth
v=30_000

#Optimal weights
ntaMVP1 = inv.mvp(return1,f=250)['w']
nta15= inv.targetP(return1,0.15)['w']

In [ ]:
#optimal weights for the MVP NTA Portfolio
ntaMVP1

array([ 0.74264428, -0.01524857, -0.08163824,  0.15621429,  0.07963346,
        0.11839479])

In [ ]:
#Optimal weights for portfolio with the target expected return of 15 percent per annum
nta15

array([ 0.76247679,  0.01081955, -0.11340901,  0.02548826,  0.17586993,
        0.13875448])

In [ ]:
#Vector of money invested into each asset
vntaMVP1 = ntaMVP1*v
vnta15= nta15*v

In [ ]:
vntaMVP1

array([22279.32825347,  -457.45703002, -2449.14728847,  4686.428556  ,
        2389.0038734 ,  3551.84363562])

In [ ]:
vnta15

array([22874.30365582,   324.58658067, -3402.27031045,   764.64769614,
        5276.09796727,  4162.63441055])

In [ ]:
#vector of innitial stock prices
ntap0 = ntaprices.iloc[0,:6]
ntap0

KO       58.526867
MSFT    251.115845
NKE     107.430824
PFE      48.074242
SBUX     69.402290
XOM      86.907921
Name: 2022-05-18 00:00:00, dtype: float64

In [ ]:
#Number of shares
nMVPnta1= vntaMVP1/ntap0
n15nta1 = vnta15/ntap0

In [ ]:
#number of stocks for MVP NTA is
nMVPnta1

KO      380.668391
MSFT     -1.821697
NKE     -22.797436
PFE      97.483151
SBUX     34.422551
XOM      40.869044
Name: 2022-05-18 00:00:00, dtype: float64

In [ ]:
#number of stocks for portfolio with target expected return of 15% annually
n15nta1

KO      390.834242
MSFT      1.292577
NKE     -31.669405
PFE      15.905559
SBUX     76.021958
XOM      47.897066
Name: 2022-05-18 00:00:00, dtype: float64

In [ ]:
#the number of shares that we need to hold over time, we can calculate the value of our portfolio on each date in testing sample simply by multiplying previous two
VntaMVP = ntaprices.iloc[:,:6]@nMVPnta1
V15nta = ntaprices.iloc[:,:6]@n15nta1

### Graphical Comparation of Static Strategy for NTA portfolio

In [ ]:
fignta1 = go.Figure()
fignta1.add_trace(
    go.Scatter(x=ntaprices.index,
               y=VntaMVP,
               line=dict(color='Blue',width=3),
               name='MVP NTA portfolio')
)
fignta1.add_trace(
    go.Scatter(
        x=ntaprices.index,
        y=V15nta,
        line=dict(color='Crimson',width=3),
        name='Target NTA 15% pa')
)
fignta1.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing 2 Static Strategy for NTA portfolio',
               x = 0.5,
               y = 0.87,
               font=dict(size=25,color='Navy'))
)

fignta1.show()

In [ ]:
V15nta[-1]-VntaMVP[-1]

2716.0072753957465

15% NTA Target porfolio outperform MVP NTA portfolio for 9.05%

In [ ]:
ntaprices.iloc[-1,:6]

KO       62.179375
MSFT    313.336609
NKE     116.206444
PFE      36.345070
SBUX    105.129402
XOM     103.967728
Name: 2023-05-17 00:00:00, dtype: float64

In [ ]:
#Weights on the last day of the trade
nMVPnta1*ntaprices.iloc[-1,:6]/VntaMVP[-1]

KO      0.742915
MSFT   -0.017916
NKE    -0.083150
PFE     0.111204
SBUX    0.113583
XOM     0.133364
dtype: float64

In [ ]:
#initial Weights
ntaMVP1

array([ 0.74264428, -0.01524857, -0.08163824,  0.15621429,  0.07963346,
        0.11839479])

### Dynamic Strategy for NTA portfolio

In [ ]:
#initial wealth
v1=30_000

#Optimal weights
ntaMVP = inv.mvp(return1,f=250)['w']
nta_15= inv.targetP(return1,0.15)['w']

In [ ]:
ret1 = ntaprices.iloc[:,:6].pct_change()

In [ ]:
ret1.dropna()

,KO,MSFT,NKE,PFE,SBUX,XOM
Date,,,,,,
2022-05-19,-0.019608,-0.003700,-0.026790,0.004960,0.009119,0.005405
2022-05-20,0.016333,-0.002291,0.014656,0.035933,0.020297,0.007900
2022-05-23,0.030830,0.032032,0.005833,0.007814,0.000409,0.022099
2022-05-24,0.018454,-0.003952,-0.012335,0.010023,-0.009534,0.005432
2022-05-25,0.000781,0.011170,0.008482,0.005617,-0.003300,0.020127
...,...,...,...,...,...,...
2023-05-11,0.005669,-0.007044,-0.010445,-0.008181,-0.007610,-0.018099
2023-05-12,0.003915,-0.003676,-0.016446,-0.006120,0.004639,-0.000095
2023-05-15,-0.002652,0.001586,-0.003161,-0.005087,0.004900,0.001907


In [ ]:
# Ex-post reteurns on MVP portfolio if we keep the weights fixed
retntaMVP= (ret1@ntaMVP)
# Ex-post reteurns on nta15 portfolio if we keep the weights fixed
ret15P = (ret1@nta15)

In [ ]:
retntaMVP[0]=0
ret15P[0]=0

In [ ]:
VntaMVP1 = (1+retntaMVP).cumprod()*v1
V15P1= (1+ret15P).cumprod()*v1

### Graphical Comparation of Dynamic Strategy for NTA portfolio

In [ ]:
figntad = go.Figure()
figntad.add_trace(go.Scatter(
    x=ntaprices.index,
    y=VntaMVP1,
    line=dict(color='Blue', width=3),
    name='MVP')
)
figntad.add_trace(go.Scatter(
    x=ntaprices.index,
    y=V15P1,
    line=dict(color='Crimson', width=3),
    name='Target 15%')
)
figntad.update_layout(
    xaxis=dict(title_text='Date', zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor= 'Black'),
    title=dict(text = 'Comparison of two dynamic strategies with constant weights',
               x=0.5,
               y=0.87,
               font=dict(size=25,
                         color='Navy'))
)

In [ ]:
V15P1[-1]-VntaMVP1[-1]

2811.079762807989

## CAPM model for NTA portfolio

In [ ]:
#Downloading data and prepare it for next steps
SP500=yf.download('^GSPC', start='2022-05-18',end='2023-05-18')['Adj Close']


[*********************100%%**********************]  1 of 1 completed


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Preparing data for rmrf

sp500 = pd.read_csv('SP500.csv', index_col=0)
retsp500 = sp500.pct_change().dropna()
retsp500

,Adj Close
Date,
2022-05-19,-0.005834
2022-05-20,0.000146
2022-05-23,0.018555
2022-05-24,-0.008121
2022-05-25,0.009451
...,...
2023-05-11,-0.001697
2023-05-12,-0.001583
2023-05-15,0.002958


In [ ]:
rmrf1 = retsp500-0.03

In [ ]:
rmrf1

,2022-05-18 00:00:00,2022-05-19 00:00:00,2022-05-20 00:00:00,2022-05-23 00:00:00,2022-05-24 00:00:00,2022-05-25 00:00:00,2022-05-26 00:00:00,2022-05-27 00:00:00,2022-05-31 00:00:00,2022-06-01 00:00:00,...,2023-05-05 00:00:00,2023-05-08 00:00:00,2023-05-09 00:00:00,2023-05-10 00:00:00,2023-05-11 00:00:00,2023-05-12 00:00:00,2023-05-15 00:00:00,2023-05-16 00:00:00,2023-05-17 00:00:00,DTB3
DATE,,,,,,,,,,,,,,,,,,,,,
2022-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# TA Portfolio

### Data Preparation

In [ ]:
#Technical Analysis portfolio start
taportf = ['AZN', 'NVDA', 'NVS', 'RIVN', 'CCL', 'NCLH']

In [ ]:
taprices = yf.download(taportf,start='2022-05-18',end='2023-05-18')['Adj Close']
#taprices.to_csv('taprices.csv',encoding='utf-8-sig')
#files.download('taprices.csv')

[*********************100%%**********************]  6 of 6 completed


In [ ]:
rets2= pd.read_csv('taprices.csv',index_col=0)

### From portfolio returns (Portfolio as an asset)

In [ ]:
#returns
rets2= taprices.pct_change().dropna()
rets2

#Weights
w2=[0.13,0.22,0.04,0.37,0.12,0.12]

#return on portfolio
rp2 = rets2@w2
rp2

Date
2022-05-19    0.014090
2022-05-20   -0.016602
2022-05-23    0.001928
2022-05-24   -0.047915
2022-05-25    0.030667
                ...   
2023-05-11   -0.013151
2023-05-12   -0.021508
2023-05-15    0.025521
2023-05-16    0.000646
2023-05-17    0.029414
Length: 250, dtype: float64

In [ ]:
inv.expRet(rets2,250,True),inv.annualize_vol(rets2,250)

(AZN     0.205298
 CCL    -0.202312
 NCLH   -0.097484
 NVDA    0.783339
 NVS     0.204596
 RIVN   -0.508611
 dtype: float64,
 AZN     0.235327
 CCL     0.737278
 NCLH    0.705900
 NVDA    0.555384
 NVS     0.196501
 RIVN    0.731755
 dtype: float64)

In [ ]:
er2 = inv.expRet(rp2)
print(er2)
vol2= inv.annualize_vol(rp2)
print(vol2)

0.2892442355037811
0.42503385257917314


### From stocks' returns (Portfolio as collection of assets)

In [ ]:
#Expected Return on portfolio
per2=inv.per(w2,rets2,250,True)
per2

0.2316343654909691

In [ ]:
pV2 = inv.pV(w2,rets2,250,True)
pV2

0.42334385044853723

###MVP

In [ ]:
mvp2 = inv.mvp(rets2,250,True)
mvp2

{'w': array([ 0.24931158, -0.05201152,  0.05462125,  0.0201236 ,  0.69997385,
         0.02798125]),
 'er': 0.20112504209198118,
 'vol': 0.18616522597525045}

### Manual optimization

In [ ]:
res2 = sco.minimize(lambda w2: pV2,
                        [0.13,0.22,0.04,0.37,0.12,0.12],
                       constraints=[dict(type='eq',fun=lambda w2:sum(w2)-1)])
res2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.42334385044853723
       x: [ 1.300e-01  2.200e-01  4.000e-02  3.700e-01  1.200e-01
            1.200e-01]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
    nfev: 7
    njev: 1

Expected return is 42.34%

In [ ]:
target2=inv.targetP(rets2,0.45)
target2

{'w': array([ 0.19373611, -0.10761361,  0.06282586,  0.27633806,  0.69960966,
        -0.12489608]),
 'er': 0.45,
 'vol': 0.22170065319023574}

In [ ]:
result2 = sco.minimize(lambda w2: pV2,
                       [0.13,0.22,0.04,0.37,0.12,0.12],
                       constraints=[dict(type='eq',fun=lambda w2: sum(w2)-1),
                                    dict(type='eq',fun=lambda w2: inv.per(w2,rets2,250,True)-0.45)])
result2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.42334385044853723
       x: [ 1.614e-01  1.608e-01  4.063e-03  5.299e-01  1.512e-01
           -7.353e-03]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
    nfev: 8
    njev: 1

In [ ]:
tares2 = sco.minimize(lambda w2: pV2,
                       [0.13,0.22,0.04,0.37,0.12,0.12],
                        bounds=[[0,0.39]]*6,
                       constraints=[dict(type='eq',fun=lambda w2: sum(w2)-1),
                                    dict(type='eq',fun=lambda w2: inv.per(w2,rets2)-0.4)])
tares2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.42334385044853723
       x: [ 2.491e-01  8.294e-02  5.301e-02  3.900e-01  2.250e-01
            0.000e+00]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00]
    nfev: 8
    njev: 1

In [ ]:
tares2.x

array([0.24907137, 0.08294372, 0.05300779, 0.39      , 0.22497712,
       0.        ])

### Efficient Frontier for TA poftolio

In [ ]:
taEF = inv.EF(rets2,plot='full')
taEF

In [ ]:
figta=inv.EF(rets2,Range=[0.01,1.5],plot='full')
figta

In [ ]:
rfa2 = pdr.get_data_fred('DTB3',start='2022-05-18',end='2023-05-18')/100
rfa2.head()
rfa2.dropna()

,DTB3
DATE,
2022-05-18,0.0102
2022-05-19,0.0104
2022-05-20,0.0101
2022-05-23,0.0107
2022-05-24,0.0105
...,...
2023-05-12,0.0508
2023-05-15,0.0506
2023-05-16,0.0507


In [ ]:
rf2 = (1+rfa2)**(1/251)-1
rf2.head()
rf2.dropna()

,DTB3
DATE,
2022-05-18,0.000040
2022-05-19,0.000041
2022-05-20,0.000040
2022-05-23,0.000042
2022-05-24,0.000042
...,...
2023-05-12,0.000197
2023-05-15,0.000197
2023-05-16,0.000197


In [ ]:
opt2 = inv.maxSharpe(rets2,rf2,f=251,com=True)
opt2

{'w': array([ 0.04405018, -0.31063093,  0.00485122,  1.        ,  0.7443998 ,
        -0.48267028]),
 'er': 1.2582810753556841,
 'vol': 0.481653968864085,
 'sharpe': 2.537897090761473}

In [ ]:
figta.add_trace(go.Scatter(x=[inv.pV(opt2['w'],rets2,n=250,vol=True)],
                            y=[inv.per(opt2['w'],rets2)],
                            marker=dict(
                                color='Crimson',
                                size=10,
                                line=dict(
                                    color='Black',
                                    width=2
                                )
                            )
                          )
)

#plot show
figta.show()

### Test for Eff Frontier in TA Portfolio

In [ ]:
#Initial Wealth
Wealth2 = 30_000

#Determined amount invested in each stock
si2=opt2['w']*Wealth2

#Number of stocks
n2=si2/taprices.iloc[0]
n2

AZN      21.462747
CCL    -673.332925
NCLH      9.153240
NVDA    177.314994
NVS     265.767977
RIVN   -530.601244
Name: 2022-05-18 00:00:00, dtype: float64

In [ ]:
#Value of portfolio each point in the time
ST2 = taprices@n2
ST2

Date
2022-05-18    30000.000000
2022-05-19    29546.483017
2022-05-20    29679.017556
2022-05-23    30878.162534
2022-05-24    31543.266669
                  ...     
2023-05-11    65629.417906
2023-05-12    65898.749756
2023-05-15    66385.598491
2023-05-16    66925.576427
2023-05-17    67576.406811
Length: 251, dtype: float64

### Static Strategy for TA portfolio

In [ ]:
#preparing data
return2 = taprices.iloc[:,:6].pct_change()
return2.dropna(inplace=True)

#initial wealth
v2=30_000

#Optimal weights
taMVP2 = inv.mvp(return2,f=250)['w']
ta30= inv.targetP(return2,0.30)['w']

In [ ]:
#optimal weights for the MVP TA Portfolio
taMVP2

array([ 0.24931158, -0.05201152,  0.05462125,  0.0201236 ,  0.69997385,
        0.02798125])

In [ ]:
#Optimal weights for portfolio with the target expected return of 15 percent per annum
ta30

array([ 0.22737622, -0.07334673,  0.0571908 ,  0.11895518,  0.70015905,
       -0.03033452])

In [ ]:
#Vector of money invested into each asset
vtaMVP2 = taMVP2*v2
vta30= ta30*v2

In [ ]:
vtaMVP2

array([ 7479.34735317, -1560.34564813,  1638.63762742,   603.70791209,
       20999.21540221,   839.43735324])

In [ ]:
vta30

array([ 6821.28645786, -2200.40187147,  1715.72411174,  3568.65546448,
       21004.77145318,  -910.03561579])

In [ ]:
#vector of innitial stock prices
tap0 = taprices.iloc[0,:6]
tap0

AZN      61.572060
CCL      13.840000
NCLH     15.900000
NVDA    169.190430
NVS      84.028160
RIVN     27.290001
Name: 2022-05-18 00:00:00, dtype: float64

In [ ]:
#Number of shares
nMVPta2= vtaMVP2/tap0
n30ta2 = vta30/tap0

In [ ]:
#number of stocks for MVP NTA is
nMVPta2

AZN     121.473074
CCL    -112.741736
NCLH    103.058973
NVDA      3.568215
NVS     249.906881
RIVN     30.759887
Name: 2022-05-18 00:00:00, dtype: float64

In [ ]:
#number of stocks for portfolio with target expected return of 30% annually
n30ta2

AZN     110.785420
CCL    -158.988573
NCLH    107.907179
NVDA     21.092537
NVS     249.973002
RIVN    -33.346852
Name: 2022-05-18 00:00:00, dtype: float64

In [ ]:
#the number of shares that we need to hold over time, we can calculate the value of our portfolio on each date in testing sample simply by multiplying previous two
VtaMVP = taprices.iloc[:,:6]@nMVPta2
V30ta = taprices.iloc[:,:6]@n30ta2

### Graphical Comparation of Static Strategy for TA portfolio

In [ ]:
figta2 = go.Figure()
figta2.add_trace(
    go.Scatter(x=taprices.index,
               y=VtaMVP,
               line=dict(color='Blue',width=3),
               name='MVP TA portfolio')
)
figta2.add_trace(
    go.Scatter(
        x=taprices.index,
        y=V30ta,
        line=dict(color='Crimson',width=3),
        name='Target TA 30% pa')
)
figta2.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing 2 Static Strategy for TA portfolio',
               x = 0.5,
               y = 0.87,
               font=dict(size=25,color='Navy'))
)

figta2.show()

In [ ]:
V30ta[-1]-VtaMVP[-1]

3200.3754840210604

Target TA portfolio with 10.66% outperform MVP TA portfolio

In [ ]:
taprices.iloc[-1,:6]

AZN      74.212685
CCL      11.040000
NCLH     14.350000
NVDA    301.723907
NVS     101.220001
RIVN     13.410000
Name: 2023-05-17 00:00:00, dtype: float64

In [ ]:
#Weights on the last day of the trade
nMVPta2*taprices.iloc[-1,:6]/VtaMVP[-1]

AZN     0.250178
CCL    -0.034542
NCLH    0.041042
NVDA    0.029878
NVS     0.701997
RIVN    0.011447
dtype: float64

In [ ]:
#initial Weights
taMVP2

array([ 0.24931158, -0.05201152,  0.05462125,  0.0201236 ,  0.69997385,
        0.02798125])

### Dynamic Strategy for TA portfolio

In [ ]:
#initial wealth
v2=30_000

#Optimal weights
taMVP = inv.mvp(return2,f=250)['w']
ta_30= inv.targetP(return2,0.30)['w']

In [ ]:
ret2 = taprices.iloc[:,:6].pct_change()
ret2.dropna()

,AZN,CCL,NCLH,NVDA,NVS,RIVN
Date,,,,,,
2022-05-19,0.019267,-0.018786,0.003145,0.010981,0.011063,0.085013
2022-05-20,0.025875,-0.033137,-0.040752,-0.025111,0.008686,-0.023303
2022-05-23,0.003021,-0.001523,-0.013072,0.012220,0.014426,-0.032158
2022-05-24,0.003614,-0.102975,-0.119868,-0.044029,0.013449,-0.052161
2022-05-25,0.002401,0.035714,0.033108,0.050823,-0.007397,0.027139
...,...,...,...,...,...,...
2023-05-11,-0.001870,-0.023832,-0.029830,-0.010628,-0.002026,-0.019135
2023-05-12,0.000000,-0.040039,-0.030747,-0.008328,-0.002707,-0.067197
2023-05-15,0.003479,0.048830,0.035499,0.021630,-0.000194,0.041053


In [ ]:
# Ex-post reteurns on MVP portfolio if we keep the weights fixed
retTaMVP= (ret2@taMVP)
# Ex-post reteurns on Ta 30 portfolio if we keep the weights fixed
ret30P = (ret2@ta30)

In [ ]:
retTaMVP[0]=0
ret30P[0]=0

In [ ]:
VtaMVP1 = (1+retTaMVP).cumprod()*v2
V30P1= (1+ret30P).cumprod()*v2

### Graphical Comparation of Dynamic Strategy for TA portfolio

In [ ]:
figtad = go.Figure()
figtad.add_trace(go.Scatter(
    x=taprices.index,
    y=VtaMVP1,
    line=dict(color='Blue', width=3),
    name='MVP')
)
figtad.add_trace(go.Scatter(
    x=taprices.index,
    y=V30P1,
    line=dict(color='Crimson', width=3),
    name='Target 30%')
)
figtad.update_layout(
    xaxis=dict(title_text='Date', zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor= 'Black'),
    title=dict(text = 'Comparison of two dynamic strategies with constant weights',
               x=0.5,
               y=0.87,
               font=dict(size=25,
                         color='Navy'))
)

In [ ]:
V30P1[-1]-VtaMVP1[-1]

3474.01089070303

# Graphs

## Graphical Comparation of Effective Frontier

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=ST1.index,y= ST1, name = 'NTA Portfolio'))
fig.add_trace(go.Scatter(x=ST2.index,y=ST2, name = 'TA Portfolio'))


fig.show()

## Graphical Comparison of Static Strategy

In [ ]:
figtot = go.Figure()
figtot.add_trace(
    go.Scatter(x=ntaprices.index,
               y=VntaMVP1,
               line=dict(color='Blue',width=3),
               name='MVP NTA portfolio')
)
figtot.add_trace(
    go.Scatter(
        x=ntaprices.index,
        y=V15nta,
        line=dict(color='Navy',width=3),
        name='Target NTA 15% pa')
)
figtot.add_trace(
    go.Scatter(x=taprices.index,
               y=VtaMVP,
               line=dict(color='Red',width=3),
               name='MVP TA portfolio')
)
figtot.add_trace(
    go.Scatter(
        x=taprices.index,
        y=V30ta,
        line=dict(color='Crimson',width=3),
        name='Target TA 30% pa')
)
figtot.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing NTA and TA Static Strategy',
               x = 0.5,
               y = 0.87,
               font=dict(size=25,color='Navy'))
)

figtot.show()

## Graphical Comparison of Trading Strategy

In [ ]:
figtot1 = go.Figure()
figtot1.add_trace(
    go.Scatter(x=ntaprices.index,
               y=VntaMVP,
               line=dict(color='Blue',width=3),
               name='MVP NTA portfolio')
)
figtot1.add_trace(
    go.Scatter(
        x=ntaprices.index,
        y=V15P1,
        line=dict(color='Navy',width=3),
        name='Target NTA 15% pa')
)
figtot1.add_trace(
    go.Scatter(x=taprices.index,
               y=VtaMVP1,
               line=dict(color='Red',width=3),
               name='MVP TA portfolio')
)
figtot1.add_trace(
    go.Scatter(
        x=taprices.index,
        y=V30P1,
        line=dict(color='Crimson',width=3),
        name='Target TA 30% pa')
)
figtot1.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing NTA and TA Dynamic Strategy',
               x = 0.5,
               y = 0.87,
               font=dict(size=25,color='Navy'))
)

figtot1.show()